 <img src="Events.png" width="320"> 
 
# Events

The whole canvas and named components can respond to mouse and
keyboard events.  Canvases and reference frames provide methods
for converting event pixel offsets into x and y coordinates
relative to the frame of reference.  Events over a named object
provide the properties of the named object as `event.object_info`.

In [ ]:
from jp_doodle import dual_canvas
from IPython.display import display

In [ ]:
# In this demonstration we do most of the work in Javascript.

demo = dual_canvas.DualCanvasWidget(width=320, height=220)
display(demo)

demo.js_init("""

// A floating tooltip (not a canvas object)
var tooltip = $("<div>tooltip here</div>").appendTo(element);
tooltip.css({
    position: "absolute",
    width: "140px",
    height: "auto",
    background: "#ddd",
    font: "12px sans-serif",
    opacity: 0
});

// Draw some named elements on the canvas.
// A filled yellow circle (disk) named "Colonel Mustard
element.circle({name: "Colonel Mustard", x:100, y:150, r:90, color:"yellow"});

// A filled red rectangle named "Miss Scarlett"
element.rect({name: "Miss Scarlett", x:100, y:130, w:100, h:20, color: "red"});

// An unfilled white circle named "Mrs. White"
element.circle({
    name: "Mrs. White", x:100, y:150, r:58, fill:false, 
    color:"white", lineWidth: 14});
    
// An unfilled blue rectangle named Mrs. Peacock
element.rect({
    name: "Mrs. Peacock", x:40, y:110, w:100, h:20,
    color: "blue", lineWidth: 10, degrees:70, fill:false});

// A line segment named "Professor Plum".
element.line({
    name: "Professor Plum", x1:190, y1:100, x2:10, y2:200,
    color:"purple", lineWidth: 20});
    
// A click indicator circle, initially hidden
element.circle({
    name: "click indicator", x:0, y:0, r:3, hide:true
});
    
var say_on_click = function(phrase) {
    return function (event) {
        var object_name = event.object_info.name;
        var element_offset = element.visible_canvas.offset();
        var canvas_location = element.event_model_location(event);
        var pixel_offset = element.event_pixel_location(event);
        var name = event.canvas_name;
        element.change_element("click indicator", {hide: false, x: canvas_location.x, y: canvas_location.y});
        tooltip.offset({
            left: pixel_offset.x + element_offset.left + 10,
            top: pixel_offset.y + element_offset.top + 10,
        })
        tooltip.html("<b>"+object_name + "</b> says <em>" + phrase + "</em>");
        tooltip.css({opacity: 0.8});
    };
};
// Attach some click handlers to the objects.
element.on_canvas_event("click", say_on_click("By Jove!"), "Colonel Mustard");
element.on_canvas_event("click", say_on_click("My Word!"), "Miss Scarlett");
element.on_canvas_event("click", say_on_click("Goodness!"), "Mrs. Peacock");
element.on_canvas_event("click", say_on_click("By Gad!"), "Professor Plum");

// Background click makes the dialog and click indicator disappear
element.on_canvas_event("click", function() {
    tooltip.css({opacity: 0});
    element.change_element("click indicator", {hide: true});
    });

// Fit the figure into the available space
element.fit(null, 10);

$("<button>Silence Col. Mustard</button>")
.appendTo(element)
.click(function() {
    element.off_canvas_event("click", "Colonel Mustard");
});

var saved_events = null;  // keep events for re-enabling later.

$("<button>Disable events</button>")
.appendTo(element)
.click(function() {
    saved_events = element.reset_events();
});

$("<button>Re-enable events</button>")
.appendTo(element)
.click(function() {
    if (saved_events) {
        element.restore_events(saved_events);
    }
});
""")

In [ ]:
demo.save_pixels_to_png_async("Events.png")